## Workflow: **Routing**


A workflow where user input is classified and directs to a specific task (this can be a specific LLM or a function).

This allows you to optimize for many inputs in isolation.

<img src="../images/routing.webp" alt="Prompt Chaining" width="500">

## **Use Cases:**

- Routing easy/common questions to smaller models like Llama 3.1 8B and hard/unusual questions to more capable models like Deepseek v3 and Llama 3.3 70B to optimize cost and speed.
- Directing different types of customer service queries (general questions, refund requests, technical support) into different downstream processes, prompts, and tools.
- Different LLMs or model configurations excel at different tasks (e.g., writing summaries vs. generating code). Using a router, you can automatically detect the user's intent and send the input to the best-fit model.
- Evaluating whether a request meets certain guidelines or triggers specific filters (e.g., checking if content is disallowed). Based on the classification, forward it to the appropriate next LLM call or step.
- If one model's output doesn't meet a certain confidence threshold or fails for some reason, route automatically to a fallback model.

In [ ]:
%pip install openai pydantic --upgrade

In [6]:
import os
from typing import Literal
from pydantic import BaseModel
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"

In [16]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o"

In [25]:
def classify_as_spam(user_query: str) -> bool:
    spam_prompt = f'''You are a spam classifier that takes in a user query and returns True if the user query is spam and False otherwise.
    The user query is: {user_query}'''
    spam_response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": spam_prompt}],
        temperature=0.0,
        max_tokens=1000,
        response_format=bool
    )
    if spam_response.choices[0].message.content is None:
        raise ValueError("Spam response is None")
    return spam_response.choices[0].message.content

def generate_summary(user_query: str) -> str:
    summary_prompt = f'''You are a summary generator that takes in a user query and returns a summary of the user query.
    The user query is: {user_query}'''
    summary_response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": summary_prompt}],
        temperature=0.0,
        max_tokens=1000,
    )
    if summary_response.choices[0].message.content is None:
        raise ValueError("Summary response is None")
    return summary_response.choices[0].message.content

In [26]:
class Router(BaseModel):
    destination: Literal["classify_as_spam", "generate_summary"]

In [27]:
def router(user_query: str) -> Router:
    router_prompt = f'''You are a router that takes in a user query and returns the destination of the user query.
    The destinations are "divide" and "multiply".
    The user query is: {user_query}'''
    
    router_response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": router_prompt}],
        temperature=0.0,
        max_tokens=1000,
        response_format=Router
    )

    if router_response.choices[0].message.content is None or router_response.choices[0].message.parsed.destination is None:
        raise ValueError("Router response is None")
    
    return router_response.choices[0].message.parsed

In [28]:
try:
    user_query = '''I want to generate a summary of this: 
    A data engineer delivers the designs set by more senior members of the data engineering community.

    At this role level, you will:
    implement data flows to connect operational systems, data for analytics and business intelligence (BI) systems
    document source-to-target mappings
    re-engineer manual data flows to enable scaling and repeatable use
    support the build of data streaming systems
    write ETL (extract, transform, load) scripts and code to ensure the ETL process performs optimally
    develop business intelligence reports that can be reused
    build accessible data for analysis
    '''

    router_choice = router(user_query)
    if router_choice.destination == "classify_as_spam":
        print(classify_as_spam(user_query))
    elif router_choice.destination == "generate_summary":
        print(generate_summary(user_query))
except Exception as e:
    print(e)

A data engineer at this role level is responsible for implementing data flows between operational, analytics, and BI systems, documenting source-to-target mappings, re-engineering manual data flows for scalability, supporting data streaming systems, writing optimized ETL scripts, developing reusable business intelligence reports, and building accessible data for analysis.


In [30]:
print(router_choice.destination)

generate_summary
